# 1. Initial Setup








*   connect to Google Drive
*   specify project folder
*   check GPU and project version

a. Connect to Google Drive. Here we allow the colab notebook access to your google drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

b. Specify the **folder name** below. This also needs to be specified in the "Config" section

In [ ]:
folder_name = "project1"  

#don't edit me:
from IPython.display import Audio
scripts_dir = "/content/drive/My Drive/GAN/" + folder_name +"/scripts"
bleep = scripts_dir + '/beep.wav'
%cd $scripts_dir



c. This cell is to check our runtime type and project version.

  The statement should say "using **cuda** for computation". If not go to Runtime < change runtime type and select "GPU".


In [ ]:
%run version_ctrl.py
%run check_gpu.py


#2. Data Processing & Training Images



 



This is for processing raw data (images and videos) in our Google Drive into a form that our model can use. This only needs to be done **once**.  

a. Add Images and Videos into "data/raw/" folder (formats allowed: jpeg, png, mp4, avi, mkv)

b.  Adjust **colour** and **delta_t** paramters and run the cell



> To convert images to B&W set **colour=0**, otherwise leave **colour=1** 


> Frames are taken from the video every **delta_t** seconds and processed as images.


> Images are center cropped to a **square format** and compressed to **128x128 pixels**.




In [ ]:
colour = 1            # 0 for B&W  ; 1 for colour

delta_t = 1           # time interval between video frame grabs (seconds) 

%run process_data.py {colour} {delta_t}
Audio(bleep, autoplay=True)

**View** the processed images:

In [ ]:
%run dataloader_init.py
Audio(bleep, autoplay=True)


If you are happy with the processed images, **delete** the contents of the "**data/raw**" folder. 

You can then add more images/videos here and repeat steps a and b
 if you want to add to your training images dataset.

#3. Config.








Here we specify the variables for our model.

Once specified we run the cell and this writes the variables to a file

If changes are made we need to **restart runtime** (Runtime < restart runtime) and rerun the cell **b** in **1. Initial Setup** 

In [ ]:

%%writefile config.py

#BASIC SETTINGs:

folder_name = "project1"

image_size = 64   #size of images created in pixels ie 64x64 pixels. Values available: {8,16,32,64,128}

nc = 3            # Number of channels in the training images. For color images this is 3. For black and white use 1

nz = 100          #Size of z latent vector (generator input) 
                  #for image_size = 32,64,128 use 50,100,400 respectively

ngf = 64          # Size of feature maps in generator;
                  # set to image_size, unless image_size =128, then set to 64

ndf = 64          # Size of feature maps in discriminator;
                  # set to image_size. Unless image_size =128, then set to 32


#ADVANCED SETTINGS:


workers = 8         # Number of workers for dataloader

batch_size = 128    # Batch size during training

lr = 0.0002         # Learning rate for optimizers

beta1 = 0.5         # Beta1 hyperparam for Adam optimizers

ngpu = 1            # Number of GPUs available. Use 0 for CPU mode.

lsf = 0.2           # label softening factor  fake= 0:lsf; real = (1-lsf): 1


#DON'T EDIT! ...

root_dir = "/content/drive/My Drive/GAN/" + folder_name +"/"
dataset_dir = root_dir + "data/processed/"
rawdata_dir = root_dir + "data/raw/"
video_dir = root_dir + "video/"
model_dir = root_dir + 'model/'
script_dir = root_dir + 'scripts/'
checkpoint_path = root_dir + 'checkpoint/checkpoint.tar'



# 4. Train your Model




Here we train our model for a specified number of **epochs**.

An epoch is when the model has cycled through all our training images once.

The more we train our model the better it becomes (usually) at generating images 

**START or RESTART** training:




In [ ]:
epochs = 25

%run train.py False {epochs}
Audio(bleep, autoplay=True)

**CONTINUE** training:



In [ ]:
epochs = 100

%run train.py True {epochs}
Audio(bleep, autoplay=True)

plot a **graph** to check the quality of the training: 

In [ ]:
# plot training loss
%run plot_loss.py

# 5. See Generated Images

Show **64 samples** of images generated by the model:

In [ ]:
%run plot_fakes.py 15

Show a **single** sample:





In [ ]:
%run plot_fake.py 8

# 6. Make a Movie!

Here we make a movie by sequencing our generated images at 24 frames per second.

Our movie consists of **transitioning** between generated samples (like those in section 5.) 

The **smoothing** variable defines how gradual (or "smooth") this transition will be.





In [ ]:
movie_name = "movie1"

fps = 24          # frames per second
smoothing = 3.0     # duration between samples (seconds)
duration= 30     # movie length in (seconds)

%run make_movie.py {movie_name} {fps} {smoothing} {duration}

Once you're happy with your movie, you can increase the resolution x16 by running the below:

In [ ]:
#run only once
!pip install ISR

In [ ]:
%run make_movie_isr.py {movie_name} {fps} {smoothing} {duration}

# Save/ Load Model

If you are happy with the results, give your model a name and save it. You can then load at a later date and continue to train or make movies with it.

In [ ]:
#SAVE MODEL

model_name = "model1"

%run save_model.py {model_name}

In [ ]:
#LOAD MODEL

model_name = "model1"

%run load_model.py {model_name}